<a href="https://colab.research.google.com/github/RoseManoj/movie-recommender-system/blob/main/movie_recomm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
import pandas as pd
movies=pd.read_csv('movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


search engine


to clean title

In [136]:
import re
def clean_title(title):
    return re.sub(r'[^a-zA-Z0-9 ]', '', title).lower()


In [137]:
movies['clean_title'] = movies['title'].apply(lambda x: re.sub(r'[^a-zA-Z0-9 ]', '', x).lower())



In [138]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii 1995
...,...,...,...,...
62418,209157,We (2018),Drama,we 2018
62419,209159,Window of the Soul (2001),Documentary,window of the soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,bad poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),a girl thing 2001


term frequency
3 movie name


1.   the
2.   harry the potter
3.   the harry

each word if present then 1 else 0

then inverse doc frequency

tf*idf









In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(ngram_range=(1,2))
tfidf=vectorizer.fit_transform(movies["clean_title"])

search function

In [140]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = similarity.argsort()[-5:][::-1]
    return movies.iloc[indices][['movieId', 'title', 'genres', 'clean_title']]





In [141]:
#indices

bulding interactive search box

In [142]:
# @title
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='',
    description='Movie Title:',
    placeholder='Type a movie name...',
    disabled=False
)

movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"].lower()
        if len(title) > 3:
            display(search(title))

movie_input.observe(on_type, names='value')

display(movie_input, movie_list)


Text(value='', description='Movie Title:', placeholder='Type a movie name...')

Output()

Reading rating of data

In [143]:
ratings=pd.read_csv("ratings.csv")


In [144]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [145]:
ratings.dtypes

,0
userId,int64
movieId,int64
rating,float64
timestamp,int64


finding users  who like the same movie

In [146]:
movie_id = 1



In [147]:
merged = pd.merge(rec_percentages, movies, left_index=True, right_on="movieId")
# Merge the recommendation percentages with movie details
merged = pd.merge(rec_percentages, movies, left_index=True, right_on="movieId")

# Show top 10 recommendations as one table
merged_result = merged[['movieId', 'title', 'genres', 'similar', 'all','score','clean_title']].head(10)
merged_result


,movieId,title,genres,similar,all,score,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.000000,0.125844,7.946323,toy story 1995
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,0.295498,0.054186,5.453383,toy story 2 1999
2264,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,0.124685,0.025316,4.925186,bugs life a 1998
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,0.138161,0.035445,3.897906,toy story 3 2010
580,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,0.233674,0.068117,3.430480,aladdin 1992
587,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,0.198949,0.060514,3.287671,beauty and the beast 1991
33,34,Babe (1995),Children|Drama,0.158226,0.052696,3.002602,babe 1995
4780,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,0.210647,0.071444,2.948410,monsters inc 2001
1047,1073,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical,0.143418,0.049202,2.914882,willy wonka the chocolate factory 1971
729,745,Wallace & Gromit: A Close Shave (1995),Animation|Children|Comedy,0.108322,0.037362,2.899227,wallace gromit a close shave 1995


In [148]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages= pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]


In [149]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]  # <- fixed column name
            display(find_similar_movies(movie_id))  # make sure this function is defined

movie_name_input.observe(on_type, names='value')
display(movie_name_input, recommendation_list)


Text(value='Toy Story', description='Movie Title:')

Output()